# Graph-Based Model Checking for EMMAA

This notebook explores the ut

In [165]:
import json
import pickle
from collections import defaultdict
import boto3
import pandas as pd
import networkx as nx
from indra.statements import *
from emmaa.util import get_s3_client

In [117]:
pd.set_option('display.max_rows', 500)

Load the latest model manager and test results JSON for the EMMAA Ras Machine model:

In [180]:
s3 = get_s3_client()
# Get the Model Manager containing the assembled INDRA Statements
s3_obj = s3.get_object(Bucket='emmaa', Key='results/rasmachine/latest_model_manager.pkl')
mm = pickle.loads(s3_obj['Body'].read())
# Get the Test results JSON
s3_obj = s3.get_object(Bucket='emmaa', Key='results/rasmachine/results_2019-04-30-17-51-24.json')
res = json.load(s3_obj['Body'])

Get the assembled statements from the Model Manager and build a NetworkX DiGraph based on agent names:

In [183]:
stmts = mm.model.assembled_stmts
len(stmts)

5453

In [184]:
edges = []
provenance = defaultdict(list)
for stmt in stmts:
    if not len(stmt.agent_list()) == 2:
        continue
    subj, obj = stmt.agent_list()
    edges.append((subj.name, obj.name))
    if isinstance(stmt, Complex):
        edges.append((obj.name, subj.name))
    provenance[(subj.name, obj.name)].append(stmt)
g = nx.DiGraph()
g.add_edges_from(edges)    

Get the tests and results from the result JSON and compile a list of (source, target) test pairs along with results obtained from the ModelChecker:

In [185]:
g_tests = []
for result in res[1:]:
    code = result['result_json']['result_code']
    if result['english_path']:
        mc_path_len = len(result['english_path'][0])
    else:
        mc_path_len = 0
    tj = result['test_json']
    test_stmt = stmts_from_json([tj])[0]
    if len(test_stmt.agent_list()) == 2:
        subj, obj = test_stmt.agent_list()
        if subj.name != obj.name:
            g_tests.append((tj['type'], subj.name, obj.name, code, mc_path_len))

Using the directed graph compiled from the model statements we look for shortest paths connecting the pairs of test nodes and build a Pandas DataFrame with the results:

In [188]:
rows = []
for stmt_type, subj, obj, code, mc_path_len in g_tests:
    sp_text = ''
    g_path_len = 0
    if subj not in g:
        g_code = 'SUBJECT_NODE_NOT_FOUND'
    elif obj not in g:
        g_code = 'OBJECT_NODE_NOT_FOUND'
    else:
        g_code = 'PATH_FOUND'
        try:
            sp = nx.shortest_path(g, subj, obj)
            g_path_len = len(sp) - 1
            sp_text = ' -> '.join(sp)
        except nx.NetworkXNoPath:
            g_code = 'NO_PATH'
    rows.append((stmt_type, subj, obj, code, mc_path_len, g_code, g_path_len, sp_text))
df = pd.DataFrame.from_records(rows, columns=['test_stmt_type', 'subj', 'obj', 'mc_code',
                                              'mc_path_len', 'g_code', 'g_path_len', 'shortest_path'])

Let's look at some statistics from the results. First, the cases where both the ModelChecker and the Graph found a path:

In [189]:
both_path = df[(df.mc_code == 'PATHS_FOUND') & (df.g_code == 'PATH_FOUND')]

In [190]:
print("Number of MC tests passed: %d" % len(df[df.mc_code == 'PATHS_FOUND']))
print("Number of tests passed by both MC and G: %d" % len(both_path))
both_path

Number of MC tests passed: 117
Number of tests passed by both MC and G: 117


,test_stmt_type,subj,obj,mc_code,mc_path_len,g_code,g_path_len,shortest_path
0,Activation,AKT1,MTOR,PATHS_FOUND,1,PATH_FOUND,1,AKT1 -> MTOR
2,Activation,AKT1,RPS6KB2,PATHS_FOUND,2,PATH_FOUND,1,AKT1 -> RPS6KB2
15,Activation,BRAF,ELK1,PATHS_FOUND,2,PATH_FOUND,2,BRAF -> MAPK1 -> ELK1
17,Activation,BRAF,MAPK1,PATHS_FOUND,1,PATH_FOUND,1,BRAF -> MAPK1
18,Activation,BRAF,MAPK3,PATHS_FOUND,1,PATH_FOUND,1,BRAF -> MAPK3
21,Activation,CASP8,CASP3,PATHS_FOUND,1,PATH_FOUND,1,CASP8 -> CASP3
31,Activation,CDK2,PPP1CA,PATHS_FOUND,1,PATH_FOUND,1,CDK2 -> PPP1CA
48,Activation,E2F1,TP53,PATHS_FOUND,3,PATH_FOUND,1,E2F1 -> TP53
58,Activation,EGFR,ERBB2,PATHS_FOUND,1,PATH_FOUND,1,EGFR -> ERBB2
60,Activation,EGFR,HRAS,PATHS_FOUND,2,PATH_FOUND,1,EGFR -> HRAS


Does it ever happen that the ModelChecker finds a path where there is none in the graph? Apparently not, which is reassuring because the paths found by the ModelChecker should be a strict subset of those in the directed graph:

In [152]:
df[(df.mc_code == 'PATHS_FOUND') & (df.g_code != 'PATH_FOUND')]

,stmt_type,subj,obj,mc_code,mc_path_len,g_code,shortest_path


As an additional sanity check, we look for cases where the ModelChecker yields a *shorter* path than the shortest path in the graph, which should not happen (and doesn't):

In [193]:
df[(df.mc_code == 'PATHS_FOUND') & (df.mc_path_len < df.g_path_len)]

,test_stmt_type,subj,obj,mc_code,mc_path_len,g_code,g_path_len,shortest_path


Next, how often is it that tests failed by the ModelChecker are passed in the Graph:

In [159]:
g_only_path = df[(df.mc_code != 'PATHS_FOUND') & (df.g_code == 'PATH_FOUND')]
mc_fail = len(df[df.mc_code != 'PATHS_FOUND'])
g_only = len(g_only_path)
ratio = g_only / mc_fail
print("Number of MC tests failed: %d" % mc_fail)
print("Number of tests failed by MC and passed by G: %d" % g_only)
print("Ratio: %.3f" % ratio)
g_only_path


Number of MC tests failed: 750
Number of tests failed by MC and passed by G: 392
Ratio: 0.523


,stmt_type,subj,obj,mc_code,mc_path_len,g_code,shortest_path
16,Activation,BRAF,ERK,NO_PATHS_FOUND,0,PATH_FOUND,BRAF -> ROS1 -> ERK
22,Activation,CASP8,NFKB1,NO_PATHS_FOUND,0,PATH_FOUND,CASP8 -> MAPK1 -> NFKB1
28,Activation,CD28,AKT,OBSERVABLES_NOT_FOUND,0,PATH_FOUND,CD28 -> GRB2 -> ROS1 -> MTOR -> AKT
30,Activation,CDK2,E2F,NO_PATHS_FOUND,0,PATH_FOUND,CDK2 -> CDK4 -> E2F
32,Activation,CDK4,E2F,NO_PATHS_FOUND,0,PATH_FOUND,CDK4 -> E2F
34,Activation,CDK6,E2F,NO_PATHS_FOUND,0,PATH_FOUND,CDK6 -> CDK4 -> E2F
36,Activation,CDKN1A,PAK1,NO_PATHS_FOUND,0,PATH_FOUND,CDKN1A -> PAK1
56,Activation,EGFR,AKT,OBSERVABLES_NOT_FOUND,0,PATH_FOUND,EGFR -> MTOR -> AKT
57,Activation,EGFR,ELK1,NO_PATHS_FOUND,0,PATH_FOUND,EGFR -> MAPK1 -> ELK1
59,Activation,EGFR,ERK,NO_PATHS_FOUND,0,PATH_FOUND,EGFR -> ROS1 -> ERK


So approximately half of the tests that fail in the ModelChecker pass in the directed graph. Inspection of the specific cases highlights a few recurring scenarios:
